# Pre-requisite installations if needed

In [ ]:
!pip install oauth2client
!pip install google-api-python-client
!pip install ipywidgets
!pip install plotly
!pip install tqdm

# --------------------------------------------------------------------------------------------------

In [1]:
import argparse
import pandas as pd
import json
import os
import ipywidgets as widgets
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

from tqdm.notebook import tqdm, trange

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
from helper_functions import initialize_analyticsreporting, get_report, print_response, VIEW_ID, next_date_interval, progress_bar_counter

pio.renderers.default = "iframe"

analytics = initialize_analyticsreporting()

# --------------------------------------------------------------------------------------------------

#### The cell below will render the widgets needed to select the items in the graph. This cell only needs to be run ONCE (to show the widgets only). After display, you don't have to run this cell. The report/graph will include the end date.

# --------------------------------------------------------------------------------------------------

In [7]:
feature = widgets.Dropdown(
    options=[
        'New/Returning Users',
        'App Launched - OS', 
        'App Launched - SODA', 
        'Manage Dataset - Create Empty Dataset', 
        'Manage Dataset - Rename Existing Dataset', 
        'Manage Dataset - Change PI owner', 
        'Manage Dataset - Add User Permission',
        'Manage Dataset - Add/Edit Subtitle',
        'Manage Dataset - Add/Edit Description', 
        'Manage Dataset - Upload Banner Image', 
        'Manage Dataset - Assign License',
        'Manage Dataset - Upload Local Dataset', 
        'Manage Dataset - Change Dataset Status',
        'Prepare Metadata - Add Airtable account',
        'Prepare Metadata - Add DDD',
        'Prepare Metadata - Create Submission',
        'Prepare Metadata - Create dataset_description',
        'Generate Dataset - Local',
        'Generate Dataset - Blackfynn',
        'Generate Dataset - Pennsieve', 
        'Manifest Files Created - Blackfynn', 
        'Disseminate Dataset - Share with Curation Team', 
        'Disseminate Dataset - Share with Consortium',
        'Disseminate Dataset - Pre-publishing Review'],
    value='New/Returning Users',
    description='Option:',
    disabled=False,
)

start_date = widgets.DatePicker(description='Start Date:', disabled=False)
end_date = widgets.DatePicker(description='End Date:', disabled=False)

update_interval = widgets.Dropdown(options=['Daily', 'Weekly', 'Monthly'], description='Update Interval:', disabled=False)

display(feature, start_date, end_date, update_interval)

Dropdown(description='Option:', options=('New/Returning Users', 'App Launched - OS', 'App Launched - SODA', 'M…

DatePicker(value=None, description='Start Date:')

DatePicker(value=None, description='End Date:')

Dropdown(description='Update Interval:', options=('Daily', 'Weekly', 'Monthly'), value='Daily')

# --------------------------------------------------------------------------------------------------

#### The cell below is a basic function that uses the widgets in the cell above to create a graph. If the widgets are not showing, run the widget cell. You don't have to run it again after selecting a value. Changing the value of the dropdown will dynamically change the value of the variable in the next cell.

# --------------------------------------------------------------------------------------------------

In [22]:
dt = start_date.value
ds = end_date.value

data = []
column_headers = []
file_name = ""
bar_counter = 0

if update_interval.value == "Daily":
    bar_counter = progress_bar_counter(dt, ds, "Daily")
    start = end = dt
    column_headers = ['Day', 'Frequency']
    file_name = "daily"
if update_interval.value == "Weekly":
    bar_counter = progress_bar_counter(dt, ds, "Weekly")    
    start = dt - timedelta(days=dt.weekday())
    end = start + timedelta(days=6)
    column_headers = ['Week', 'Frequency']
    file_name = "weekly"
if update_interval.value == "Monthly":
    bar_counter = progress_bar_counter(dt, ds, "Monthly")
    start = end = dt
    column_headers = ['Month', 'Frequency']
    file_name = "monthly"
    
for i in trange(bar_counter):
    if start <= ds:
        query = {
            'reportRequests': [
            {
                'viewId': VIEW_ID,
                'dateRanges': [{'startDate': start.strftime('%Y-%m-%d'), 'endDate': end.strftime('%Y-%m-%d')}],
                'metrics': [{'expression': 'ga:totalEvents'}],
                'dimensions': [{'name': 'ga:eventAction'}]
            }]
        }
        cell_data_date = ""
        if update_interval.value == "Daily":
            cell_data_date = start.strftime("%d %b, %Y")     
        if update_interval.value == "Weekly":
            cell_data_date = start.strftime("%d %b, %Y") + " - " + end.strftime("%d %b, %Y")    
        if update_interval.value == "Monthly":
            cell_data_date = start.strftime("%b %Y")
        
        response = response_rows = []
        response = get_report(analytics, query)
        if "rows" in response["reports"][0]["data"]:
            response_rows = response["reports"][0]["data"]["rows"]
        else:
            response_rows = []
            cell_data = [cell_data_date, 0]
            data.append(cell_data)
        
        if response_rows != []:
            response_present = False
            for res in response_rows:
                if res["dimensions"][0] == feature.value:
                    cell_data = [cell_data_date, int(res["metrics"][0]["values"][0])]
                    data.append(cell_data)
                    response_present = True
            if response_present == False:
                cell_data = [cell_data_date, 0]
                data.append(cell_data)
        
        start, end = next_date_interval(start, end, update_interval.value)
        
folder_path = os.path.join("result_csv", "graph_data")
Path(folder_path).mkdir(parents=True, exist_ok=True)

df = pd.DataFrame(data, columns = column_headers)
result_path = os.path.join(folder_path, file_name + "_graph-" + dt.strftime("%d %b, %Y") + " - " + ds.strftime("%d %b, %Y") + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

# action_column = df.iloc[:, 0]
# frequency_column = df.iloc[:, 1]
# x_markers = pd.Series(action_column).array
# y_markers = pd.Series(frequency_column).array
# y_markers = y_markers.astype(int)
df.astype({'Frequency': 'int32'}).dtypes

fig = None

if update_interval.value == "Daily":
    fig = px.line(
        df, 
        x = "Day", 
        y = "Frequency",
        labels = {
            "Day": "Date",
            "Frequency": "Frequency",
        },
        title = update_interval.value + " Chart for '" + feature.value + "': " + dt.strftime("%d %b, %Y") + " - " + ds.strftime("%d %b, %Y"),
        render_mode = "auto"
    )
if update_interval.value == "Weekly":
    fig = px.line(
        df, 
        x = "Week", 
        y = "Frequency",
        labels = {
            "Day": "Date",
            "Frequency": "Frequency",
        },
        title = update_interval.value + " Chart for '" + feature.value + "': " + dt.strftime("%d %b, %Y") + " - " + ds.strftime("%d %b, %Y"),
        render_mode = "auto"
    )
if update_interval.value == "Monthly":
    fig = px.line(
        df, 
        x = "Month", 
        y = "Frequency",
        labels = {
            "Day": "Date",
            "Frequency": "Frequency",
        },
        title = update_interval.value + " Chart for '" + feature.value + "': " + dt.strftime("%d %b, %Y") + " - " + ds.strftime("%d %b, %Y"),
        render_mode = "auto"
    )

fig.show()



['20 Apr, 2021', 3]
['21 Apr, 2021', 2]

['23 Apr, 2021', 1]
['24 Apr, 2021', 1]

['27 Apr, 2021', 1]

             Day  Frequency
0   13 Apr, 2021          0
1   13 Apr, 2021          0
2   14 Apr, 2021          0
3   14 Apr, 2021          0
4   15 Apr, 2021          0
5   15 Apr, 2021          0
6   16 Apr, 2021          0
7   16 Apr, 2021          0
8   17 Apr, 2021          0
9   17 Apr, 2021          0
10  18 Apr, 2021          0
11  18 Apr, 2021          0
12  19 Apr, 2021          2
13  19 Apr, 2021          0
14  20 Apr, 2021          3
15  20 Apr, 2021          4
16  20 Apr, 2021          0
17  21 Apr, 2021          2
18  21 Apr, 2021          7
19  21 Apr, 2021          0
20  22 Apr, 2021          2
21  22 Apr, 2021          0
22  23 Apr, 2021          1
23  23 Apr, 2021          4
24  23 Apr, 2021          0
25  24 Apr, 2021          1
26  24 Apr, 2021          0
27  24 Apr, 2021          0
28  25 Apr, 2021          0
29  25 Apr, 2021          0
30  26 Apr, 2021          1


# --------------------------------------------------------------------------------------------------

In [8]:
# Date format in 'YYYY-MM-DD'
# You can also use relative dates for simplicity
# start_date = "50daysAgo"
# end_date = "today"
# end_date = "yesterday"
start_date = "2021-01-23"
end_date = "2021-04-23"

### Get a report of all the events that occured with a status of either "success" or "error" within a given time frame

In [10]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
both_data, success_data, error_data = [], [], []

for res in response_rows:
    both_cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
    both_data.append(both_cell_data)
    if res["dimensions"][0] == "Success":
        success_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        success_data.append(success_cell_data)
    if res["dimensions"][0] == "Error":
        error_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        error_data.append(error_cell_data)

folder_path = os.path.join("result_csv", "status_count")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(both_data, columns = ['Status', 'Action', 'Values'])
result_path = os.path.join(folder_path, "Both-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(success_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Success-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(error_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Error-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all app launches within a given time frame

In [11]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
app_launch_os, app_launch_soda = [], []

for res in response_rows:
    if res["dimensions"][0] == "App Launched - OS":
        app_launch_os_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_os.append(app_launch_os_cell_data)
    if res["dimensions"][0] == "App Launched - SODA":
        app_launch_soda_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_soda.append(app_launch_soda_cell_data)

folder_path = os.path.join("result_csv", "app_launched")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(app_launch_os, columns = ['OS', 'Values'])
result_path = os.path.join(folder_path, "os-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(app_launch_soda, columns = ['SODA Version', 'Values'])
result_path = os.path.join(folder_path, "soda_version-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all unique users within a given time frame

In [12]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:users'}],
        'dimensions': [{'name': 'ga:userType'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
data = []

for res in response_rows:
    cell_data = [res["dimensions"][0], res["metrics"][0]["values"][0]]
    data.append(cell_data)
    
folder_path = os.path.join("result_csv", "users")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(data, columns = ['Type', 'Values'])
result_path = os.path.join(folder_path, "users-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all dataset names for a specific action for a given time frame

In [13]:
def number_of_actions(start_date, end_date, category, action):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][0] == category:
            if res["dimensions"][1] == action:
                cell_data = [res["dimensions"][2], res["metrics"][0]["values"][0]]
                data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Dataset_name', 'Values'])
    result_path = os.path.join(folder_path, action + "-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting the names of datasets where an action is applicable
## all responses go to the custom folder
# number_of_actions(start_date, end_date, <type>, <action_name>)
number_of_actions(start_date, end_date, "Success", "Manage Dataset - Create Empty Dataset")

### Get a report of all actions done on a specific dataset for a given time frame

In [14]:
def dataset_actions(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][2].find(dataset_name) != -1 or res["dimensions"][1].find(dataset_name) != -1:
            cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Action', 'Values'])
    result_path = os.path.join(folder_path, dataset_name + "(actions)-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the actions for a specific dataset
## all responses go to the custom folder
# dataset_actions(start_date, end_date, <Dataset_name>)
dataset_actions(start_date, end_date, "test-ps-SODA")

### Get the number of files and the size of all datasets that was uploaded through SODA for a given time frame

In [15]:
def dataset_statistics(start_date, end_date):
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:uniqueEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    
    for res in response_rows:
#         print(res)
        if res["dimensions"][1] == "Upload Local Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Upload Local Dataset - size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, "dataset_statistics-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all details for upload to Pennsieve for a specific time period
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date)
dataset_statistics(start_date, end_date)

### Get the number of files and the size of a specific dataset that was uploaded through SODA

In [16]:
def num_of_files_folders_in_dataset(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []
    
    for res in response_rows:
        if res["dimensions"][1].find("Upload Local Dataset") != -1:
            if res["dimensions"][1].find(dataset_name) != -1:
                if res["dimensions"][1].find('Number of Files') != -1:
                    cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
                    data.append(cell_data)
                if res["dimensions"][1].find('- size') != -1:
                    cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
                    data.append(cell_data)
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:eventValue'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    res["metrics"][0]["values"][0]
    for res in response_rows:
        if res["dimensions"][1].find("Generate Dataset") != -1:
            if res["dimensions"][2].find(dataset_name) != -1:
                if res["dimensions"][1] == 'Generate Dataset - Number of Files':
                    cell_data = [res["dimensions"][0], res["metrics"][0]["values"][0], 0]
                    data.append(cell_data)
                if res["dimensions"][1] == 'Generate Dataset - Size':
                    cell_data = [res["dimensions"][0], 0, res["metrics"][0]["values"][0]]
                    data.append(cell_data)
                
                    
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, dataset_name + "(details)-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the number of files and size for a specific dataset
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date, <Dataset_name>)
num_of_files_folders_in_dataset(start_date, end_date, "test-ps-SODA")